# URL 链接

一些实现代理或以其他方式使用另一个文件系统实现，例如本地缓存远程文件，即使用远程实现查找存在的文件，但实际上在访问时打开本地副本。其他示例包括从 Dask 工作节点读取，该节点可以看到客户端无法看到的文件系统，以及访问正在从另一个后端读取的 zip 文件。

在这种情况下，可以像在实现文档字符串中指定的那样精确指定参数，对于 dask 的情况，可能是这样的：

```python
of = fsspec.open('dask://bucket/key', target_protocol='s3', target_options={'anon': True})
```

作为一种速记法，特别是在有多个跳转的情况下，可以使用特殊的分隔符 `"::"` 来“链接”URL。要传递给每个引用实现的参数按 URL 中包含的协议名称键入。以下是与上面一行等效的内容：

```python 
of = fsspec.open('dask::s3://bucket/key', s3={'anon': True})
```

一些更复杂的情况：

```python
of = fsspec.open_files('zip://*.csv::simplecache::gcs://bucket/afile.zip',
                       simplecache={'cache_storage': '/stored/zip/files'},
                       gcs={'project': 'my-project'})
```

从 Google 读取一个 zip 文件，将其存储在本地，并提供对其中包含的 CSV 文件的访问。相反地，

```python
of = fsspec.open_files('simplecache::zip://*.csv::gcs://bucket/afile.zip',
                       simplecache={'cache_storage': '/stored/csv/files'},
                       gcs={'project': 'my-project'})
```

读取相同的 zip 文件，但提取 CSV 文件并将它们存储在本地缓存中。

对于开发人员：这种“链接”方法的工作原理是将传递给 `open_*` 的参数格式化为 `target_protocol` （一个简单的字符串）和 `target_options` （一个字典），如果需要特定文件，还可以选择性地提供 `fo`（目标路径）。为了使实现能够像这样成功地链接，它必须准确地查找这些命名参数。